## DATA MINING PROJECT  - Customer Segmentation

**Group EZ**  
**Group Members:**  
Beatriz Carmo - 20220685  
Marta Antunes - 20221094  
Tomás Corte-Real - 20221639

**Msc:** Data Science and Advanced Analytics - Major in Bussiness Ananlytics @ NOVA IMS  
**Course:** Data Mining - 2022/2023

GitHub Repository Link: https://github.com/BeatrizC25/DataMiningProject22-23.git

# Table of Contents

# 1. Imports

In [1]:
%autosave 90

import warnings
warnings.filterwarnings('ignore')

from os.path import join
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import AgglomerativeClustering
from scipy.cluster.hierarchy import dendrogram

sns.set()

Autosaving every 90 seconds


## 2. Loading and Processing Data

In [2]:
#loading file into a dataframe
df_original=pd.read_sas('a2z_insurance.sas7bdat')

In [3]:
#create a copy of the original dataframe
df=df_original.copy()

In [4]:
#check how many rows and columns the dataset has
print('The dataset has a total of', df.shape[0], 'rows and', df.shape[1], 'columns.')

The dataset has a total of 10296 rows and 14 columns.


**Notice there are 10296 columns in the dataset and only 10290 costumers.**

In [22]:
df.head()

,CustID,FirstPolYear,BirthYear,EducDeg,MonthSal,GeoLivArea,Children,CustMonVal,ClaimsRate,PremMotor,PremHousehold,PremHealth,PremLife,PremWork
0,1.0,1985.0,1982.0,b'2 - High School',2177.0,1.0,1.0,380.97,0.39,375.85,79.45,146.36,47.01,16.89
2,3.0,1991.0,1970.0,b'1 - Basic',2277.0,3.0,0.0,504.67,0.28,206.15,224.50,124.58,86.35,99.02
3,4.0,1990.0,1981.0,b'3 - BSc/MSc',1099.0,4.0,1.0,-16.99,0.99,182.48,43.35,311.17,35.34,28.34
4,5.0,1986.0,1973.0,b'3 - BSc/MSc',1763.0,4.0,1.0,35.23,0.90,338.62,47.80,182.59,18.78,41.45
5,6.0,1986.0,1956.0,b'2 - High School',2566.0,4.0,1.0,-24.33,1.00,440.75,18.90,114.80,7.00,7.67


In [6]:
#checking for ducplicates in dataset
df.duplicated(df.columns[1:]).sum()

3

In [7]:
#remove duplicated rows
df.drop_duplicates(df.columns[1:], inplace = True)
print('Now, the dataset has', df.shape[0], 'rows.')

Now, the dataset has 10293 rows.


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10293 entries, 0 to 10295
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   CustID         10293 non-null  float64
 1   FirstPolYear   10263 non-null  float64
 2   BirthYear      10276 non-null  float64
 3   EducDeg        10276 non-null  object 
 4   MonthSal       10257 non-null  float64
 5   GeoLivArea     10292 non-null  float64
 6   Children       10272 non-null  float64
 7   CustMonVal     10293 non-null  float64
 8   ClaimsRate     10293 non-null  float64
 9   PremMotor      10259 non-null  float64
 10  PremHousehold  10293 non-null  float64
 11  PremHealth     10250 non-null  float64
 12  PremLife       10189 non-null  float64
 13  PremWork       10207 non-null  float64
dtypes: float64(13), object(1)
memory usage: 1.2+ MB


In [9]:
#cheking how many missing values are in dataset
df.isna().sum()

CustID             0
FirstPolYear      30
BirthYear         17
EducDeg           17
MonthSal          36
GeoLivArea         1
Children          21
CustMonVal         0
ClaimsRate         0
PremMotor         34
PremHousehold      0
PremHealth        43
PremLife         104
PremWork          86
dtype: int64

**There are several missing values in different features.**  
This issue will be adressed and taken care of later in this analysis.

In [10]:
df.columns

Index(['CustID', 'FirstPolYear', 'BirthYear', 'EducDeg', 'MonthSal',
       'GeoLivArea', 'Children', 'CustMonVal', 'ClaimsRate', 'PremMotor',
       'PremHousehold', 'PremHealth', 'PremLife', 'PremWork'],
      dtype='object')

The dataset has both **categorical features** (EducDeg, GeoLivArea, Children) and numeric ones. Some of the categorical feautres are types as being **floats** (this is the case of GeoLivArea and Children) **instead of objects**. This can pose a future issue, therefore those categorical features must have their type corrected.

In [11]:
#correcting the type of categorical features
df['GeoLivArea']=df['GeoLivArea'].astype(object)
df['Children']=df['Children'].astype(object)

Let's take a look into the descriptive statistics! 

In [12]:
df.describe().T #this only includes numeric features

,count,mean,std,min,25%,50%,75%,max
CustID,10293.0,5147.504712,2972.180611,1.00,2574.00,5147.00,7720.00,10296.00
FirstPolYear,10263.0,1991.064893,511.342617,1974.00,1980.00,1986.00,1992.00,53784.00
BirthYear,10276.0,1968.006909,19.710742,1028.00,1953.00,1968.00,1983.00,2001.00
MonthSal,10257.0,2506.618699,1157.518390,333.00,1706.00,2502.00,3290.00,55215.00
CustMonVal,10293.0,177.929963,1946.091554,-165680.42,-9.44,187.03,399.86,11875.89
ClaimsRate,10293.0,0.742728,2.917385,0.00,0.39,0.72,0.98,256.20
PremMotor,10259.0,300.501550,211.937740,-4.11,190.59,298.61,408.30,11604.42
PremHousehold,10293.0,210.419863,352.635041,-75.00,49.45,132.80,290.05,25048.80
PremHealth,10250.0,171.553183,296.444399,-2.11,111.80,162.81,219.04,28272.00
PremLife,10189.0,41.848724,47.484264,-7.00,9.89,25.56,57.79,398.30


Now let's check the **coherence of the data**!  
Assumptions:  
 - One cannot be more than 120 years-old
 - One cannot have their 1st year policy after 2016
 - One cannot have their birth year be after their 1st year policy
 - One cannot be less than 16 and not have finished high school to have a job 
 - One cannot have finished their degree with less than 18 years-old
 - One cannot spend more money than they own  

In [30]:
#removing all inciherent rows from dataset 
df.drop(df[(df.BirthYear<1896)|(df.BirthYear>2016)].index, inplace = True)
df.drop(df[df.FirstPolYear>2016].index, inplace=True)
df.drop(df[df.BirthYear>df.FirstPolYear].index, inplace=True)
df.drop(df[(df.BirthYear>2000) & (df.MonthSal>0) & (df.EducDeg==b'1 - Basic')].index, inplace=True)
df.drop(df[(df.BirthYear>1998) & ((df.EducDeg==b'3 - BSc/MSc')|(df.EducDeg==b'4 - PhD'))].index,
                                  inplace=True)

In [29]:
#create function to compute the sum of the values of Premiums of each row
def sum_(*args):
    tot=0
    for x in args:
        tot+=x
    return tot

df.drop(df[sum_(df.PremMotor, df.PremHousehold, df.PremHealth, df.PremLife, df.PremWork)>df.MonthSal].index,
       inplace=True)

After removing incoherent rows, let's check if there are any incoherencies left.

In [43]:
print((df.BirthYear<1896).sum(), (df.BirthYear>2016).sum(), (df.FirstPolYear>2016).sum(),
(df.BirthYear>df.FirstPolYear).sum(),((df.BirthYear>2000) & (df.MonthSal>0) & (df.EducDeg==b'1 - Basic')).sum(),
((df.BirthYear>1998) & ((df.EducDeg==b'3 - BSc/MSc')|(df.EducDeg==b'4 - PhD'))).sum(),
(sum_(df.PremMotor, df.PremHousehold, df.PremHealth, df.PremLife, df.PremWork)>df.MonthSal).sum())

0 0 0 0 0 0 0


There are no incoherent values!

**Divide and conquer!**  
Let's now devide the several features into **numeric** and **non-numeric** ones will to ease future steps in this analysis.

In [44]:
#dividing features and store them into 2 lists
non_metric_feat = ["EducDeg", "GeoLivArea", "Children"]
metric_feat = df.columns.drop(non_metric_feat).to_list()